# Overview
I'm mostly familiar with databricks; however, wanted to try out building a common etl pattern in snowflake as a learning experience.

This is a basic etl pattern for fact and dimension table loads (excluding aggregates). This will assume 1 dimension and 1 fact, but the pattern is extensible to handle many tables. The only requirement would be from a scheduling standpoint dims run first and then facts based on dependency order.

While the full example is contained in this notebook, this would be split up into multiple notebooks in a true production environment. For example:
1. A notebook per table with just the create statements and any primary key assignments (e.g. create_dim_employee).
2. A notebook per table with just the etl load logic per table (e.g. load_dim_employee).
3. Various stage load processes - excluded for this demo.
4. Common functions split out into a reusable and sharable file.

## Libraries and Imports

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.row import Row
session = get_active_session()

In [ ]:
database_name = 'learning_db'
schema_name = 'dw'
src_schema_name = 'src'
etl_schema_name = 'etl'

## Basic Env Exploration

In [ ]:
df_tables = session.sql(f'SHOW TABLES IN SCHEMA {database_name}.{schema_name}')
df_tables.show()

## Table Setup
This would be split into another notebook, but for demo purposes including here. Let's assume we have a stage table of employee listing and pay dates from our operational database or erp. Assumption is these are already setup in our staging environment in a src schema.

### Simulate Stage Env

In [ ]:
session.sql(f"""
CREATE SCHEMA IF NOT EXISTS {database_name}.{src_schema_name};
""")

In [ ]:
session.sql(f"""
CREATE SCHEMA IF NOT EXISTS {database_name}.{etl_schema_name};
""")

In [ ]:
session.sql(f"""
SHOW SCHEMAS IN {database_name};
""")

In [ ]:
session.sql(f"""
CREATE TABLE {database_name}.{src_schema_name}.employee (
    employee_id BIGINT,
    name STRING,
    department STRING
);
""")

In [ ]:
session.sql(f"""
INSERT INTO {database_name}.{src_schema_name}.employee (employee_id, name, department)
VALUES
    (1, 'Jack', 'Finance'),
    (2, 'Jill', 'Merchandising')
""")

In [ ]:
session.sql(f"""
SELECT * FROM {database_name}.{src_schema_name}.employee
""").show()

In [ ]:
session.sql(f"""
CREATE TABLE {database_name}.{src_schema_name}.employee_pay (
    employee_id BIGINT,
    pay_date DATE,
    pay_amount NUMBER(10,2)
);
""")

In [ ]:
session.sql(f"""
INSERT INTO {database_name}.{src_schema_name}.employee_pay (employee_id, pay_date, pay_amount)
VALUES
    (1, '2025-09-30', 100.00),
    (2, '2025-09-30', 200.00)
""")

## Create Tables
Each cell would be its own file. Importantly, for the pattern to work primary keys on the natural composite key must be defined.

In [ ]:
session.sql(f"""
CREATE TABLE {database_name}.{schema_name}.dim_employee (
    dim_employee_key BIGINT AUTOINCREMENT,
    employee_id BIGINT,
    name STRING,
    department STRING,
    etl_row_hash_value STRING,
    create_username STRING,
    create_datetime TIMESTAMP_NTZ,
    last_update_username STRING,
    last_update_datetime TIMESTAMP_NTZ
);
""")

In [ ]:
session.sql(f"""
ALTER TABLE {database_name}.{schema_name}.dim_employee
ADD CONSTRAINT pk_dim_employee PRIMARY KEY (employee_id);
""")

In [ ]:
session.sql(f"""
DESCRIBE TABLE {database_name}.{schema_name}.dim_employee
""")

In [ ]:
# Keep natural dim foreign keys (employee_id) for debugging if ever needed
session.sql(f"""
CREATE TABLE {database_name}.{schema_name}.fact_employee_pay (
    fact_employee_pay_key BIGINT AUTOINCREMENT,
    dim_employee_key BIGINT,
    pay_date DATE,
    pay_amount NUMBER(10,2),
    employee_id BIGINT,
    etl_row_hash_value STRING,
    create_username STRING,
    create_datetime TIMESTAMP_NTZ,
    last_update_username STRING,
    last_update_datetime TIMESTAMP_NTZ
);
""")

In [ ]:
session.sql(f"""
ALTER TABLE {database_name}.{schema_name}.fact_employee_pay
ADD CONSTRAINT pk_fact_employee_pay PRIMARY KEY (employee_id, pay_date);
""")

In [ ]:
session.sql(f"""
DESCRIBE TABLE {database_name}.{schema_name}.fact_employee_pay
""")

## Load Pattern
This would be defined somewhere in a common access library file

In [ ]:
class TableUpdater:
    def __init__(
        self,
        table_name
    ):
        """Updater class that upserts data from etl view to edw table

        Args:
            table_name (str): Name of the table (view name will be inferred)
        """

        # Infer base object names and calc audit columns
        self.table_name = table_name
        self.current_username = 'demo_user' # in actual env: session.sql("SELECT CURRENT_USER() AS current_user").collect()[0][0]
        self.current_datetime_cst = session.sql("SELECT CONVERT_TIMEZONE('America/Los_Angeles', 'America/Chicago', CURRENT_TIMESTAMP())::TIMESTAMP_NTZ AS current_time_cst").collect()[0][0]
        self.full_table_name = f'{database_name}.{schema_name}.{self.table_name}'
        self.etl_view_name = f'{database_name}.{etl_schema_name}.vw_{self.table_name}'
        self.table_primary_key_column_name = f'{self.table_name}_key'
        self.updates_table_name = f'{database_name}.{etl_schema_name}.{self.table_name}_updates'

        # Full Column Listing
        column_listing: list[Row] = session.sql(f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE
            TABLE_CATALOG = UPPER('{database_name}')
        AND TABLE_SCHEMA = UPPER('{schema_name}')
        AND TABLE_NAME = UPPER('{self.table_name}')
        ORDER BY ORDINAL_POSITION;
        """).collect()
        column_listing = [row.COLUMN_NAME.lower() for row in column_listing]

        # Determine primary keys and join strings
        table_natural_keys_list: list[Row] = session.sql(f'SHOW PRIMARY KEYS IN TABLE {self.full_table_name}').collect()
        self.table_natural_keys_list = [row.column_name.lower() for row in table_natural_keys_list]
        self.natural_key_join_string = ' AND '.join([f'source.{natural_key_column_name} = target.{natural_key_column_name}' for natural_key_column_name in self.table_natural_keys_list])
        self.update_table_columns = [column_name for column_name in column_listing if column_name not in (self.table_primary_key_column_name, 'create_username', 'create_datetime', 'last_update_username', 'last_update_datetime')]
        self.insert_columns = [column_name for column_name in column_listing if column_name not in (self.table_primary_key_column_name)]
        self.update_hash_columns = [column_name for column_name in column_listing if column_name not in (self.table_primary_key_column_name, 'create_username', 'create_datetime') and column_name not in self.table_natural_keys_list]


    def identify_upserts(self):
        sql_string = f"""
        CREATE OR REPLACE TABLE {self.updates_table_name} AS 
        SELECT
             target.{self.table_primary_key_column_name}
            ,{','.join([f'source.{col}' for col in self.update_table_columns])}
            ,'{self.current_username}' as create_username
            ,CAST('{self.current_datetime_cst}' AS TIMESTAMP_NTZ) as create_datetime
            ,'{self.current_username}' as last_update_username
            ,CAST('{self.current_datetime_cst}' AS TIMESTAMP_NTZ) as last_update_datetime
            ,CASE 
                WHEN target.{self.table_primary_key_column_name} IS NULL THEN 'insert'
                ELSE 'update'
             END as insert_update_indicator
        FROM {self.etl_view_name} source
        LEFT JOIN {self.full_table_name} target
            ON {self.natural_key_join_string}
        WHERE
            target.{self.table_primary_key_column_name} IS NULL
        OR source.etl_row_hash_value <> target.etl_row_hash_value
        """
        print(sql_string)
        execution_results = session.sql(sql_string)
        execution_results.show()

        change_audit_sql_string = f"""
        SELECT
             SUM(CASE WHEN insert_update_indicator = 'insert' THEN 1 ELSE 0 END) AS new_records
            ,SUM(CASE WHEN insert_update_indicator = 'update' THEN 1 ELSE 0 END) AS change_records
        FROM {self.updates_table_name}
        """
        print(change_audit_sql_string)
        session.sql(change_audit_sql_string).show()


    def process_table_updates(self):
        
        sql_string = f"""
        MERGE INTO {self.full_table_name} as target
        USING {self.updates_table_name} as source
        ON source.{self.table_primary_key_column_name} = target.{self.table_primary_key_column_name}
        WHEN MATCHED AND source.insert_update_indicator = 'update'
        THEN UPDATE SET
        {', '.join([f'target.{column_name} = source.{column_name}' for column_name in self.update_hash_columns])}
        """
        print(sql_string)
        execution_results = session.sql(sql_string)
        execution_results.show()


    def process_table_inserts(self):
        
        sql_string = f"""
        MERGE INTO {self.full_table_name} as target
        USING {self.updates_table_name} as source
        ON {self.natural_key_join_string}
        WHEN NOT MATCHED AND source.insert_update_indicator = 'insert'
        THEN INSERT ({', '.join(self.insert_columns)})
        VALUES ({', '.join([f'source.{col}'for col in self.insert_columns])})
        """
        print(sql_string)
        execution_results = session.sql(sql_string)
        execution_results.show()
        
        
        

## Load Examples
Each of these would be their own notebook

### dim_employee

In [ ]:
session.sql(f"""
CREATE OR REPLACE VIEW {database_name}.{etl_schema_name}.vw_dim_employee AS
SELECT
     employee_id
    ,name
    ,department
    ,SHA1(CONCAT_WS('|',
        COALESCE(CAST(name as STRING), '|'),
        COALESCE(CAST(department as STRING), '|')
    )) AS etl_row_hash_value --all but natural composite key
FROM {database_name}.{src_schema_name}.employee
""")

In [ ]:
session.sql(f"""
SELECT * FROM {database_name}.{etl_schema_name}.vw_dim_employee
""").show()

In [ ]:
table_updater = TableUpdater(table_name = 'dim_employee')

In [ ]:
table_updater.identify_upserts()

In [ ]:
table_updater.process_table_updates()

In [ ]:
table_updater.process_table_inserts()

### fact_employee_pay

In [ ]:
session.sql(f"""
CREATE OR REPLACE VIEW {database_name}.{etl_schema_name}.vw_fact_employee_pay AS
SELECT
     e.dim_employee_key
    ,p.pay_date
    ,p.pay_amount
    ,p.employee_id
    ,SHA1(CONCAT_WS('|',
        COALESCE(CAST(p.pay_amount as STRING), '|')
    )) AS etl_row_hash_value --all but natural composite key
FROM {database_name}.{src_schema_name}.employee_pay p
INNER JOIN {database_name}.{schema_name}.dim_employee e
    ON p.employee_id = e.employee_id
""")

In [ ]:
session.sql(f"""
SELECT * FROM {database_name}.{etl_schema_name}.vw_fact_employee_pay
""").show()

In [ ]:
table_updater = TableUpdater(table_name = 'fact_employee_pay')

In [ ]:
table_updater.identify_upserts()

In [ ]:
table_updater.process_table_updates()

In [ ]:
table_updater.process_table_inserts()

### fact_employee_pay - test updates
Manually modifying some source data, make sure updates are picked up.

In [ ]:
session.sql(f"""
UPDATE {database_name}.{src_schema_name}.employee_pay
SET pay_amount = 150.00
WHERE employee_id = 1
""")

In [ ]:
session.sql(f"""
SELECT * FROM {database_name}.{etl_schema_name}.vw_fact_employee_pay
""").show()

session.sql(f"""
SELECT * FROM {database_name}.{schema_name}.fact_employee_pay
""").show()

In [ ]:
table_updater = TableUpdater(table_name = 'fact_employee_pay')

In [ ]:
table_updater.identify_upserts()

In [ ]:
table_updater.process_table_updates()

In [ ]:
table_updater.process_table_inserts()

In [ ]:
session.sql(f"""
SELECT * FROM {database_name}.{etl_schema_name}.vw_fact_employee_pay
""").show()

session.sql(f"""
SELECT * FROM {database_name}.{schema_name}.fact_employee_pay
""").show()

# Summary
This was a high level demonstration of a pattern that could be used in snowflake for updating fact and dim tables from source. As you can see in the last few sections, it's dynamic in a way that the focus is on architecture and business logic (view definition) for any custom code where all inserts/updates are handled dynamically.

However, this is just a start. Here are some additional topics that this pattern could be extended to handle:
1. Deletes: A method can be added to join the final fact table back to the view and delete any records that exist in the table but not the view. This would only be done with facts where a delete is appropriate and the view has the full dataset available.
2. Aggregates: Simple class could be added to do an aggregate update - given a view that queries facts/dims, build an aggregate either in full or dynamic view a date range that looks at updates recently made in all the facts.
3. Error handling/asserts: For production uses, the class should do things like check if tables/views exist, primary keys are defined, etc. In this example, I knew what I wanted to do - but having helpful assert error messages can assist the team in debugging quicker.
4. Email alerts/data integrity: Checking for various scenarios and flagging if they occur during table loads.
5. Sourcing data: Need a process to build out that src stage from true data rather than simulated records.
6. Dedup/source data quality: Source data should also have a natural key defined and deduplicate/warn if there are violations.
7. Type 2 dims: Usually try not to do this if possible, but there may be cases where type 2 dims make sense. The pattern could be expanded to keep track of a row effective and expiration date. Instead of doing an update, it would enddate the matching record and do an insert.
8. Probably a bunch of other stuff I'm forgetting at the moment!